In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import pudl
import pudl.constants as pc
import pudl.extract.ferc1
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

In [3]:
# For some reason these things don't stick if they're in the same cell as the
# %matplotlib inline call above, but if they're separate, they work fine.
plt.style.use('ggplot')
mpl.rcParams['figure.figsize'] = (10,6)
mpl.rcParams['figure.dpi'] = 100
pd.options.display.max_columns = 56

# TODO:
 * Create data sanity checks for other FERC Form 1 data tables.
 * Go back to debugging FERC ID Generation

In [5]:
start_year = 2006
end_year = 2017
pudl_out = pudl.output.pudltabl.PudlTabl(
    freq="MS",
    start_date=pd.datetime(start_year, 1, 1),
    end_date=pd.datetime(end_year, 12, 31))
steam_ferc1 = pudl_out.plants_steam_ferc1()
fuel_ferc1 = pudl_out.fuel_ferc1()
fbp_ferc1 = pudl_out.fbp_ferc1()

In [93]:
raw_ferc1_dfs = pudl.extract.ferc1.extract(ferc1_tables=['fuel_ferc1', 'plants_steam_ferc1'], ferc1_years=range(2004,2018))
tfr_ferc1_dfs = pudl.transform.ferc1.transform(raw_ferc1_dfs, ferc1_tables=['fuel_ferc1', 'plants_steam_ferc1'])

Extracting tables from cloned FERC Form 1 database.
    fuel_ferc1...
    plants_steam_ferc1...
Transforming dataframes from FERC 1:
    fuel_ferc1...
    plants_steam_ferc1...
        Identifying distinct large FERC plants.


/home/zane/anaconda3/envs/pudl/lib/python3.6/site-packages/pandas/core/indexing.py:1017: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]


        Categorized 12077 of 13568 (89.01%) plant records.
        Assigning FERC Plant IDs.
        Found 991 orphaned plant records.
        Found 1257 non-orphaned plant groups.
        Began with 13568 raw steam plant records.
        Ended with 13446 transformed steam plant records.


In [94]:
tfr_steam = tfr_ferc1_dfs['plants_steam_ferc1']

In [95]:
pid = np.random.choice(tfr_steam.plant_id_ferc1)
plant_name = tfr_steam.query(f'plant_id_ferc1=={pid}').plant_name.values[0]
match_cols = [ 'plant_id_ferc1', 'utility_id_ferc1', 'report_year', 'plant_name',
    'plant_type', 'construction_year', 'construction_type', 'capacity_mw', 'net_generation_mwh',
    'coal_fraction_mmbtu', 'gas_fraction_mmbtu', 'oil_fraction_mmbtu', 'nuclear_fraction_mmbtu',
    'waste_fraction_mmbtu', 'other_fraction_mmbtu',
]
tfr_steam.query(f'plant_name=="{plant_name}"')[match_cols].sort_values(['plant_id_ferc1', 'report_year'])
#tfr_steam.sample(10)[match_cols]

KeyError: "['coal_fraction_mmbtu', 'waste_fraction_mmbtu', 'other_fraction_mmbtu', 'oil_fraction_mmbtu', 'gas_fraction_mmbtu', 'nuclear_fraction_mmbtu'] not in index"

In [39]:
print(f"Fuel by Plant records: {len(fbp)}")
print(f"Plant records: {len(steam_ferc1)}")
merged = pd.merge(steam_ferc1, fbp, on=['utility_id_ferc1', 'report_year', 'plant_name'], how='inner')
print(f"Merged records: {len(merged)}")

NameError: name 'fbp' is not defined

In [50]:
merged[merged.duplicated(subset=['report_year', 'utility_id_ferc1', 'plant_name'], keep=False)]

,report_year,utility_id_ferc1,utility_id_pudl,utility_name_ferc1,plant_id_pudl,plant_id_ferc1,plant_name,asset_retirement_cost,avg_num_employees,capacity_mw,capex_equipment,capex_land,capex_per_mw,capex_structures,capex_total,construction_type,construction_year,id,installation_year,net_generation_mwh,not_water_limited_capacity_mw,opex_allowances,opex_boiler,opex_coolants,opex_electric,opex_engineering,opex_fuel,opex_misc_power,...,opex_rents,opex_steam,opex_steam_other,opex_structures,opex_transfer,peak_demand_mw,plant_capability_mw,plant_hours_connected_while_generating,plant_type,record_id,water_limited_capacity_mw,plant_dupes,coal_fraction_mmbtu,gas_fraction_mmbtu,nuclear_fraction_mmbtu,oil_fraction_mmbtu,other_fraction_mmbtu,waste_fraction_mmbtu,fuel_mmbtu,coal_fraction_cost,gas_fraction_cost,nuclear_fraction_cost,oil_fraction_cost,other_fraction_cost,waste_fraction_cost,fuel_cost,primary_fuel_by_mmbtu,primary_fuel_by_cost
161,2004,7,31,Arizona Public Service Company,193,30,four corners,327820.0,219.0,634.0,2.962889e+08,45947.0,518608.4,32135086.0,3.287978e+08,outdoor,1963.0,30,1964.0,4340642.093,560.0,0.0,15519293.0,0.0,1138887.0,795729.0,61623961.0,2582629.0,...,185605.0,5990119.0,0.0,921722.0,0.0,567.0,0.0,8780.0,None,2004_7_0_3,560.0,True,0.996513,0.003487,0.0,0.000000,0.0,0.0,6.319454e+07,0.981446,0.018554,0.0,0.000000,0.0,0.0,8.286467e+07,coal,coal
162,2004,7,31,Arizona Public Service Company,193,31,four corners,434907.0,65.0,245.0,1.344613e+08,18099.0,592652.6,10285631.0,1.451999e+08,outdoor,1969.0,31,1970.0,1597699.160,222.0,0.0,3792826.0,0.0,200792.0,177392.0,21258623.0,526082.0,...,69771.0,1680205.0,0.0,204963.0,0.0,239.0,0.0,8712.0,None,2004_7_0_4,222.0,True,0.996513,0.003487,0.0,0.000000,0.0,0.0,6.319454e+07,0.981446,0.018554,0.0,0.000000,0.0,0.0,8.286467e+07,coal,coal
163,2005,7,31,Arizona Public Service Company,193,30,four corners,324001.0,228.0,634.0,3.118552e+08,45947.0,543889.0,32600483.0,3.448256e+08,outdoor,1963.0,1640,1964.0,4456831.650,560.0,0.0,15060173.0,0.0,1191730.0,1185762.0,68011078.0,4518931.0,...,186927.0,6189554.0,0.0,807282.0,0.0,566.0,0.0,8760.0,None,2005_7_0_3,560.0,True,0.997329,0.002671,0.0,0.000000,0.0,0.0,6.488451e+07,0.977258,0.022742,0.0,0.000000,0.0,0.0,9.203323e+07,coal,coal
164,2005,7,31,Arizona Public Service Company,193,31,four corners,200906.0,66.0,245.0,1.454177e+08,18099.0,637052.8,10441273.0,1.560779e+08,outdoor,1969.0,1641,1970.0,1672277.500,222.0,0.0,3583749.0,0.0,193554.0,230376.0,23958185.0,712727.0,...,69771.0,1965885.0,0.0,119307.0,0.0,229.0,0.0,8760.0,None,2005_7_0_4,222.0,True,0.997329,0.002671,0.0,0.000000,0.0,0.0,6.488451e+07,0.977258,0.022742,0.0,0.000000,0.0,0.0,9.203323e+07,coal,coal
165,2006,7,31,Arizona Public Service Company,193,30,four corners,324001.0,239.0,634.0,3.211132e+08,45947.0,563068.7,35502450.0,3.569856e+08,None,1963.0,2686,1694.0,4227245.205,560.0,0.0,16997192.0,0.0,1204858.0,1503284.0,69864610.0,6795095.0,...,195718.0,6743794.0,0.0,1275051.0,0.0,567.0,0.0,8757.0,None,2006_7_0_3,560.0,True,0.997740,0.002260,0.0,0.000000,0.0,0.0,6.356240e+07,0.984880,0.015120,0.0,0.000000,0.0,0.0,9.601554e+07,coal,coal
166,2006,7,31,Arizona Public Service Company,193,31,four corners,200906.0,61.0,245.0,1.464842e+08,18099.0,641834.5,10546201.0,1.572495e+08,None,1969.0,2687,1970.0,1770262.100,225.0,0.0,2606019.0,0.0,194107.0,235818.0,26320428.0,1152736.0,...,73377.0,2153320.0,0.0,149789.0,0.0,470.0,0.0,8749.0,None,2006_7_0_4,225.0,True,0.997740,0.002260,0.0,0.000000,0.0,0.0,6.356240e+07,0.984880,0.015120,0.0,0.000000,0.0,0.0,9.601554e+07,coal,coal
168,2005,7,31,Arizona Public Service Company,567,32,saguaro,0.0,14.0,250.0,4.582974e+07,51490.0,195375.1,2962539.0,4.884376e+07,outdoor,1954.0,1643,1955.0,40631.000,209.0,0.0,79598.0,0.0,211732.0,12544.0,3688502.0,994585.0,...,0.0,430647.0,0.0,77358.0,0.0,192.0,0.0,750.0,steam,2005_7_1_1,209.0,True,0.000000,1.000000,0.0,0.000000,0.0,0.0,5.915182e+05,0.000000,1.000000,0.0,0.000000,0.0,0.0,3.942456e+06,gas,gas
169,2005,7,31,Ar

In [18]:
len(steam_ferc1)

13467

In [19]:
len(fbp)

10617

## Identifying Duplicates:
 * What does it mean for a plant to be non-unique?
 * Any combination of `plant_name`, `report_year`, and `utility_id_ferc1` that shows up more than once in the steam plants table is a duplicate, that probably shows up more than once in the fuel table as well.
 * There's also another source of duplicity in the fuel table, due to the simplification of the fuel types. However, this should be minor.
 * First we just want to **identify** these cases, with a flag, and see how many of them there are.
 * Don't start trying to think about how to fix or deal with them yet buddy.

In [10]:
cols_short = ['utility_id_ferc1', 'report_year', 'plant_name']
cols_long = ['utility_id_ferc1', 'report_year', 'plant_name', 'plant_id_ferc1', 'record_id']

tfr_plants = tfr_ferc1_dfs['plants_steam_ferc1'][cols_long]
tfr_dupes = tfr_plants[tfr_plants.duplicated(subset=cols_short, keep=False)]

out_plants = pudl_out.plants_steam_ferc1()[cols_long]
out_dupes = out_plants[out_plants.duplicated(subset=cols_short, keep=False)]

In [22]:
print(f"records with duplicates: {len(out_dupes)}")
print(f"deduplicated records: {len(out_dupes[cols_short].drop_duplicates(subset=cols_short))}")
print(f"duplicate plants (name/util): {len(out_dupes[['utility_id_ferc1', 'plant_name']].drop_duplicates(subset=['utility_id_ferc1', 'plant_name']))}")

records with duplicates: 1327
deduplicated records: 638
duplicate plants (name/util): 81


In [24]:
1327/638

2.079937304075235

In [29]:
81/len(out_plants[['utility_id_ferc1', 'plant_name']].drop_duplicates())

0.05202312138728324

In [65]:
tfr_dupes.query("plant_name=='lemon creek'")

,utility_id_ferc1,report_year,plant_name,plant_id_ferc1,record_id
16,3,2004,lemon creek,17,2004_3_0_1
17,3,2004,lemon creek,18,2004_3_0_2
1184,3,2005,lemon creek,17,2005_3_0_1
1185,3,2005,lemon creek,18,2005_3_0_2
2475,3,2006,lemon creek,17,2006_3_0_1
2476,3,2006,lemon creek,18,2006_3_0_2
3150,3,2007,lemon creek,17,2007_3_0_1
3151,3,2007,lemon creek,18,2007_3_0_2
4350,3,2008,lemon creek,17,2008_3_0_1
4351,3,2008,lemon creek,18,2008_3_0_2


In [3]:
"natural gas 95%% $%.2f /mmBTU" % 17

'natural gas 95% $17.00 /mmBTU'

In [64]:
out_dupes.query("plant_name=='lemon creek'")

,utility_id_ferc1,report_year,plant_name,plant_id_ferc1,record_id
162,3,2004,lemon creek,17,2004_3_0_1
163,3,2004,lemon creek,18,2004_3_0_2
164,3,2005,lemon creek,17,2005_3_0_1
165,3,2005,lemon creek,18,2005_3_0_2
166,3,2006,lemon creek,17,2006_3_0_1
167,3,2006,lemon creek,18,2006_3_0_2
168,3,2007,lemon creek,17,2007_3_0_1
169,3,2007,lemon creek,18,2007_3_0_2
170,3,2008,lemon creek,17,2008_3_0_1
171,3,2008,lemon creek,18,2008_3_0_2


In [78]:
f1_cols = [
    'respondent_id',
    'report_year',
    'spplmnt_num',
    'row_number',
    'plant_name'
]
f1_steam = pd.read_sql_table('f1_steam', pudl.extract.ferc1.connect_db(testing=False))
f1_steam.query("respondent_id==3").query("plant_name=='Lemon Creek'").query("report_year==2004")[f1_cols]


,respondent_id,report_year,spplmnt_num,row_number,plant_name
28,3,2004,0,1,Lemon Creek
29,3,2004,0,2,Lemon Creek


In [80]:
f1_fuel = pd.read_sql_table('f1_fuel', pudl.extract.ferc1.connect_db(testing=False))
f1_fuel.query("respondent_id==3").query("plant_name=='Lemon Creek'").query("report_year==2004")#[f1_cols]

,respondent_id,report_year,spplmnt_num,row_number,row_seq,row_prvlg,plant_name,fuel,fuel_unit,fuel_quantity,fuel_avg_heat,fuel_cost_delvd,fuel_cost_burned,fuel_cost_btu,fuel_cost_kwh,fuel_generaton,report_prd
76,3,2004,0,1,1,,Lemon Creek,oil,barrel,805.0,0.0,62.49,49.45,0.0,0.096,14000.0,12
77,3,2004,0,2,2,,Lemon Creek,,,0.0,0.0,0.00,0.00,0.0,0.000,0.0,12
78,3,2004,0,3,3,,Lemon Creek,,,0.0,0.0,0.00,0.00,0.0,0.000,0.0,12
79,3,2004,0,4,4,,Lemon Creek,oil,barrel,2247.0,0.0,62.49,49.44,0.0,0.185,14000.0,12
80,3,2004,0,5,5,,Lemon Creek,,,0.0,0.0,0.00,0.00,0.0,0.000,0.0,12
81,3,2004,0,6,6,,Lemon Creek,,,0.0,0.0,0.00,0.00,0.0,0.000,0.0,12


In [42]:
fuel_ferc1 = pd.read_sql_table('fuel_ferc1', pudl.init.connect_db(testing=False))
nf = pudl.transform.ferc1.fuel_by_plant_ferc1(fuel_ferc1, thresh=0.5)

### Integrate this work into the code base:
* Use that function to create a new output table for FERC.

### Integrate this kind of merge into regression notebook:

### Update the FERC plant ID generator:
* Use that function to add new fields to the FERC plant ID generator to match records better.
* Deal with the weird duplicate records that are showing up in there too, and breaking the plant generator.

### Integrate other improvements into outputs:
* Fill in the gaps in the MCOE outputs to enable time series analyses
* Add smoothed 12 month rolling averages to EIA MCOE outputs for costs, etc.

In [13]:
coal_pid = np.random.choice(coal_plants.plant_id_pudl)
coal_plants[coal_plants.plant_id_pudl==coal_pid]

NameError: name 'coal_plants' is not defined

In [ ]:
# For each plant record in the plants table, we need to be able to assign
# a fraction of heat content for that plant that came from a particlar fuel
# for filtering purposes. There are different ways that we can aggregate that
# information -- e.g. plant_id_pudl, plant_id_ferc1, (utility_id_ferc1, plant_name)
def coal_fraction(fuel_df, gb_cols=['plant_id_pudl']):
    """
    Calculate the fraction of overall heat content from coal
    per year and per gb_cols, which can be:
      - [plant_id_pudl]
      - [plant_id_ferc1]
      - [utility_id_ferc1, plant_name]
    
    Assumes fuel_df is a fuel_ferc1 dataframe.
    Cannot currently work for arbitrary fuel_type becuase not all plants
    have records for all fuel types.
    """
    gb_cols1 = ['report_year'] + gb_cols
    
    # Total heat content across all fuel types
    total_mmbtu = (
        fuel_df.groupby(gb_cols1).
        agg({'fuel_consumed_mmbtu': sum}).
        reset_index().
        rename(columns={'fuel_consumed_mmbtu': 'total_mmbtu'})
    )
    
    # Same grouping... but also broken down by fuel type
    gb_cols2 = gb_cols1 + ['fuel_type_code_pudl']
    out_df = (
        fuel_df[fuel_df.fuel_type_code_pudl=='coal'].
        groupby(gb_cols2).
        agg({'fuel_consumed_mmbtu': sum}).
        reset_index().
        merge(total_mmbtu, how='left', on=gb_cols1).
        assign(coal_fraction=lambda x: x['fuel_consumed_mmbtu'] / x['total_mmbtu']).
        rename(columns={'fuel_consumed_mmbtu': 'coal_mmbtu'}).
        drop(['fuel_type_code_pudl'], axis=1)
    )
    return out_df

In [ ]:
# Calculate total heat content of fuel by plant & year
fuel_consumed_mmbtu = (
    fuel_ferc1.loc[fuel_ferc1.report_year>=start_year,:].
    groupby(['report_year', 'plant_id_pudl']).
    agg({'fuel_consumed_mmbtu': sum}).
    reset_index()
)

coal_fraction_by_plant_name = coal_fraction(fuel_ferc1.loc[fuel_ferc1.report_year>=start_year,:], ['utility_id_ferc1', 'plant_name'])

steam_agg_ferc1 = (
    # Only interested in 2012 and later
    steam_ferc1.loc[steam_ferc1.report_year>=start_year,:].
    # This ensures we only include plants that use *some* coal
    merge(coal_fraction_by_plant_name, how='inner', on=['report_year', 'utility_id_ferc1', 'plant_name']).
    # Break out the non-fuel OpEx
    assign(opex_nonfuel=lambda x: x['opex_production_total'] - x['opex_fuel']).
    # Aggregate by plant_id_pudl
    groupby(['report_year','plant_id_pudl']).
    agg({'capacity_mw': sum,
         'net_generation_mwh': sum,
         'opex_fuel': sum,
         'opex_nonfuel': sum}).
    reset_index().
    # Bring in total heat content per pudl_plant
    merge(fuel_consumed_mmbtu, how='left', on=['report_year', 'plant_id_pudl']).
    # Calculate some other interesting metrics:
    assign(opex_fuel_per_mwh=lambda x: x['opex_fuel'] / x['net_generation_mwh'],
           opex_nonfuel_per_mwh=lambda x: x['opex_nonfuel'] / x['net_generation_mwh'],
           heat_rate_mmbtu_mwh=lambda x: x['fuel_consumed_mmbtu'] / x['net_generation_mwh'],
           capacity_factor=lambda x: x['net_generation_mwh'] / (8760*x['capacity_mw']),
           opex_total_per_mwh=lambda x: x['opex_nonfuel_per_mwh'] + x['opex_fuel_per_mwh'])
)

In [ ]:
coal_fraction_by_plant_name.sample(10)

In [ ]:
truly_bad = [507, 527, 408, 311, 23, 258, 406, 456, 592, 180, 360, 424, 605, 658]
fixes = [
    #(507,  2011, 1000), Reported fuel costs also under by ~10x
    (1117, 2011, 1000),
    (1117, 2012, 1000),
    (125,  2015, 1000),
    (200,  2015, 1000),
    (200,  2016, 1000),
    (200,  2017, 1000),
    (249,  2015, 1000),
    (249,  2016, 1000),
    (249,  2017, 1000),
    (284,  2015, 1000),
    (312,  2015, 1000),
    (607,  2015, 1000),
    (657,  2015, 10),
]

def fix_ferc1_netgen(df, fixes):
    # A few FERC 1 records have errors in their net_generation_mwh column.
    # After identifying these fixes, we need to apply them not only to
    # net_generation_mwh field, but also to the other affected fields:
    #  - opex_fuel_per_mwh
    #  - opex_nonfuel_per_mwh
    #  - heat_rate_mmbtu_mwh
    #  - capacity_factor
    #  - opex_total_per_mwh
    df_out = df.copy()
    for pid, yr, mult in fixes:
        try:
            rownum = df[(df.plant_id_pudl==pid) & (df.report_year==yr)].index[0]
        except:
            continue
        df_out.loc[rownum, 'net_generation_mwh'] = df_out.loc[rownum, 'net_generation_mwh']*mult
        df_out.loc[rownum, 'opex_fuel_per_mwh'] = df_out.loc[rownum, 'opex_fuel_per_mwh']/mult
        df_out.loc[rownum, 'opex_nonfuel_per_mwh'] = df_out.loc[rownum, 'opex_nonfuel_per_mwh']/mult
        df_out.loc[rownum, 'opex_total_per_mwh'] = df_out.loc[rownum, 'opex_total_per_mwh']/mult
        df_out.loc[rownum, 'heat_rate_mmbtu_mwh'] = df_out.loc[rownum, 'heat_rate_mmbtu_mwh']/mult
        df_out.loc[rownum, 'capacity_factor'] = df_out.loc[rownum, 'capacity_factor']*mult
    return(df_out)
steam_agg_ferc1 = fix_ferc1_netgen(steam_agg_ferc1, fixes)

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=2, constrained_layout=True)

fig.suptitle(f'FERC Form 1 Coal Plants Raw ({start_year}-{end_year})')

axs[0,0].hist(steam_agg_ferc1.opex_nonfuel_per_mwh, range=(0,100), bins=100);
axs[0,0].set_title('Non-Fuel OpEx')
axs[0,0].set_xlabel('$/MWh')
axs[0,0].set_ylabel('# of Plants')

axs[0,1].hist(steam_agg_ferc1.capacity_factor, range=(0,1), bins=100);
axs[0,1].set_title('Capacity Factors')
axs[0,1].set_xlabel('Capacity Factor')

axs[1,0].hist(steam_agg_ferc1.opex_fuel_per_mwh, range=(0,100), bins=100)
axs[1,0].set_title('Fuel OpEx')
axs[1,0].set_xlabel('$/MWh')
axs[1,0].set_ylabel('# of Plants')

axs[1,1].hist(steam_agg_ferc1.heat_rate_mmbtu_mwh, range=(0,30), bins=100)
axs[1,1].set_title('Heat Rates')
axs[1,1].set_xlabel('mmBTU/MWh')

plt.savefig('raw_ferc_hist.png')

plt.show();

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=1, constrained_layout=True, sharey=True)
fig.suptitle(f'FERC Form 1 Coal Plants Raw ({start_year}-{end_year})')

axs[0].scatter(steam_agg_ferc1.capacity_factor, steam_agg_ferc1.opex_fuel_per_mwh, s=steam_agg_ferc1.capacity_mw/10, alpha=0.1)
axs[0].set_yscale('log')
axs[0].set_xlabel('Capacity Factor')
axs[0].set_xlim(-0.02,1)
axs[0].set_ylabel('$/MWh')
axs[0].set_ylim(1e0,1e4)
axs[0].set_title('Fuel Cost vs. Capacity Factor')

axs[1].scatter(steam_agg_ferc1.capacity_factor, steam_agg_ferc1.opex_nonfuel_per_mwh, s=steam_agg_ferc1.capacity_mw/10, alpha=0.1)
axs[1].set_yscale('log')
axs[1].set_xlabel('Capacity Factor')
axs[1].set_xlim(-0.02,1)
axs[1].set_ylim(1e0,1e5)
axs[1].set_title('Non-Fuel Cost vs. Capacity Factor')

plt.savefig('mcoe_vs_capfac.png')

plt.show();

In [ ]:
eia_cols = [
    'report_date',
    'plant_id_eia',
    'plant_id_pudl',
    'unit_id_pudl',
    'generator_id',
    'plant_name',
    'utility_id_eia',
    'utility_id_pudl',
    'utility_name',
    'capacity_factor',
    'capacity_factor_rolling',
    'fuel_cost_per_mwh',
    'fuel_cost_per_mwh_rolling',
    'fuel_type_code_pudl',
    'heat_rate_mmbtu_mwh',
    'heat_rate_mmbtu_mwh_rolling',
    'capacity_mw',
    'net_generation_mwh',
    'net_generation_mwh_rolling',
    'total_fuel_cost',
    'total_fuel_cost_rolling',
    'fuel_cost_per_mmbtu',
    'fuel_cost_per_mmbtu_rolling',
    'total_mmbtu',
    'total_mmbtu_rolling',
]

ferc_cols = [
    'report_year',
    'plant_id_pudl',
    'opex_fuel_per_mwh',
    'opex_nonfuel_per_mwh',
    'heat_rate_mmbtu_mwh',
    'capacity_factor',
    #'capacity_mw',
    #'net_generation_mwh',
    #'opex_fuel',
    #'opex_nonfuel',
    #'fuel_consumed_mmbtu',
]

## Convert EIA data to rolling 12 month averages.
The EIA 923 monthly data is noisy because of variation in plant output. This makes it a little hard to compare directly to the annual FERC data. There are also missing data. We can use rolling sums & forward/back filling to create a smoothed, weighted, rolling averages for the ratios of interest.
### Values of Interest:
 * `heat_rate_mmbtu_mwh`
 * `fuel_cost_per_mwh`
 * `fuel_cost_per_mmbtu`
### Original Values:
 * `net_generation_mwh`
 * `total_fuel_cost`
 * `total_mmbtu`
### For each EIA/PUDL generation unit, we need to:
 * Create a complete time index that spans the time between the first and last records.
 * Merge the real data onto that complete time index
 * Fill the gaps in static 860 data with values from the same year
 * Create a rolling sum for each of the original values
 * Calculate new rolling versions of the values of interest (which are pairwise combinations of original values)

In [ ]:
def rolling_mcoe(mcoe_df, win_type='triang', window=12, min_periods=6):
    """
    Calculate rolling sums & averages of key quantities in MCOE outputs.

    The monthly EIA data is noisy and has some gaps for a variety of
    reasons. Calculating rolling annual weighted averages helps smooth
    out a bunch of irregularities, and create a better curve for
    comparison and combination with FERC Form 1 data.
    
    The target quantities that we want to re-calculate are:
     * fuel_cost_per_mwh
     * fuel_cost_per_mmbtu
     * heat_rate_mmbtu_mwh
     * capacity_factor
    
    These quantities depend on the previously calculated:
     * net_generation_mwh
     * total_fuel_cost
     * total_mmbtu
     
    In addition, we need to ensure that the time series is complete,
    i.e. that all the expected timesteps exist, even if they have no
    data associated with them, for the purpose of calculating the
    rolling sums & averages.
    """
    fdr = (
        pd.date_range(start=mcoe_df.report_date.min(),
                      end=mcoe_df.report_date.max(),
                      freq='MS', name='report_date').
        to_frame().
        assign(tmp=1)
    )
    pgids = (
        mcoe_df[['plant_id_eia', 'generator_id']].
        drop_duplicates().
        assign(tmp=1)
    )
    gb = (
        pd.merge(fdr, pgids).
        drop('tmp', axis=1).
        merge(mcoe_df, on=['report_date', 'plant_id_eia', 'generator_id'], how='left').
        set_index('report_date').
        groupby(['plant_id_eia', 'generator_id'])
    )
    rolling_df = (
        gb.rolling(12, win_type='triang', min_periods=6, center=True).
        agg({
            'net_generation_mwh': sum,
            'total_fuel_cost': sum,
            'total_mmbtu': sum
        }).
        reset_index().
        rename(columns={
            'net_generation_mwh': 'net_generation_mwh_rolling',
            'total_fuel_cost': 'total_fuel_cost_rolling',
            'total_mmbtu': 'total_mmbtu_rolling'
        }).
        merge(mcoe_df, how='left', on=['plant_id_eia', 'generator_id', 'report_date']).
        assign(
            heat_rate_mmbtu_mwh_rolling=lambda x: x['total_mmbtu_rolling']/x['net_generation_mwh_rolling'],
            fuel_cost_per_mwh_rolling=lambda x: x['total_fuel_cost_rolling']/x['net_generation_mwh_rolling'],
            fuel_cost_per_mmbtu_rolling=lambda x: x['total_fuel_cost_rolling']/x['total_mmbtu_rolling'],
            capacity_factor_rolling=lambda x: x['net_generation_mwh_rolling']/(8760*x['capacity_mw'])
        ).
        set_index('report_date')
    )
    
    return rolling_df


In [ ]:
rolling_df = rolling_mcoe(mcoe)

In [ ]:
test_pid = np.random.choice(rolling_df.plant_id_eia.unique())
#test_pid = 470
test_plant =  rolling_df[rolling_df.plant_id_eia==test_pid]
test_gid = np.random.choice(test_plant.generator_id.unique())
#test_gid = '3'

test_gen = rolling_df[
    (rolling_df.plant_id_eia == test_pid) &
    (rolling_df.generator_id == test_gid)
]

plant_name = test_gen.plant_name.dropna().unique()[0]

plt.plot(test_gen.index, test_gen.fuel_cost_per_mmbtu_rolling, label="12 month rolling")
plt.plot(test_gen.index, test_gen.fuel_cost_per_mmbtu, label="monthly")
plt.ylabel("Fuel Costs ($/mmBTU)")
plt.title(f"{plant_name} {test_gid} Fuel Costs")
plt.legend()
plt.savefig('Test_RollHeatCost.png')
plt.show()

plt.plot(test_gen.index, test_gen.fuel_cost_per_mwh_rolling, label="12 month rolling")
plt.plot(test_gen.index, test_gen.fuel_cost_per_mwh, label="monthly")
plt.ylabel("Fuel Costs ($/MWh)")
plt.title(f"{plant_name} {test_gid} Fuel Costs")
plt.legend()
plt.savefig('Test_RollMWhCost.png')
plt.show()

plt.plot(test_gen.index, test_gen.heat_rate_mmbtu_mwh_rolling, label="12 month rolling")
plt.plot(test_gen.index, test_gen.heat_rate_mmbtu_mwh, label="monthly")
plt.ylabel("Heat Rate (mmBTU/MWh)")
plt.title(f"{plant_name} {test_gid} Heat Rate")
plt.legend()
plt.savefig('Test_RollHeatRate.png')
plt.show()

plt.plot(test_gen.index, test_gen.capacity_factor_rolling, label="12 month rolling")
plt.plot(test_gen.index, test_gen.capacity_factor, label="monthly")
plt.ylabel("Capacity Factor")
plt.title(f"{plant_name} {test_gid} Capacity Factor")
plt.legend()
plt.savefig('Test_RollCapFac.png')
plt.show()

## Merging FERC & EIA together
* FERC is annual, EIA is monthly
* Before we kept EIA monthly b/c some years had bad (NA) months and aggregating to annual meant losing the whole year.
* This means that the merge has a *lot* of FERC data duplication
* Annual records are broadcast across all months of EIA
* PUDL Plant ID level aggregation is broadcast across all unit_id_pudl values in EIA
* Different sampling scales (both time and generation unit granularity) means only ratio are comparable
* Can compare cost / MWh but not absolute cost, heat rate but not fuel heat content.
* Need to figure out a way to plot this data for a single plant such that it's visually clear how well EIA & FERC values match
* Can do a scatter plot of the same values from both datasets (perfect match = diagonal line)
* Can plot the same values in both datasets against time on the x-axis

In [ ]:
rolling_df = rolling_df.reset_index()
plant_ids_2017 = rolling_df[(rolling_df.report_date.dt.year==end_year) &
                            (rolling_df.fuel_type_code_pudl=='coal')].plant_id_pudl.unique()
mcoe_both = (
    # Keep only the relevant mcoe_cols
    rolling_df.loc[:,eia_cols].reset_index().
    # Keep only record pertaining to plants that reported in 2017
    loc[(rolling_df.plant_id_pudl.isin(plant_ids_2017)) &
        (rolling_df.fuel_type_code_pudl=='coal'),:].
    #drop(['fuel_type_code_pudl'], axis=1).
    # Create an EIA report year to merge on
    assign(report_year=lambda x: x['report_date'].dt.year).
    rename(columns={'fuel_cost_per_mwh':'opex_fuel_per_mwh'}).
    # Merge with FERC on year and plant_id_pudl
    merge(steam_agg_ferc1[ferc_cols], how='left', on=['report_year', 'plant_id_pudl'], suffixes=('_eia', '_ferc')).
    drop('index', axis=1)
)

In [ ]:
# choose a random plant:
pid=np.random.choice(mcoe_both.plant_id_pudl.unique())
mcoe_test = (
    mcoe_both.loc[mcoe_both.plant_id_pudl==pid,:].
    sort_values(['report_date', 'unit_id_pudl']).
    assign(opex_total_per_mwh_ferc=lambda x: x['opex_fuel_per_mwh_ferc'] +x['opex_nonfuel_per_mwh'])
)
plt.plot(mcoe_test.report_date, mcoe_test.opex_fuel_per_mwh_ferc, label="FERC Fuel", lw=3, color='black', ls='dashed')
plt.plot(mcoe_test.report_date, mcoe_test.opex_nonfuel_per_mwh, label="FERC NonFuel", lw=3, color='gray', ls='dashed')
plt.plot(mcoe_test.report_date, mcoe_test.opex_total_per_mwh_ferc, label="FERC Total", lw=5, color='black')

for unit_id in mcoe_test.unit_id_pudl.unique():
    mcoe_unit = mcoe_test[mcoe_test.unit_id_pudl==unit_id]
    for gen_id in mcoe_unit.generator_id.unique():
        eia_gen = mcoe_unit[mcoe_unit.generator_id==gen_id]
        plt.plot(eia_gen.report_date, eia_gen.fuel_cost_per_mwh_rolling, label=f"EIA Gen {gen_id} Fuel")
    #plt.plot(mcoe_unit.report_date, mcoe_unit.opex_fuel_per_mwh_eia, label=f"EIA Unit {unit_id} Fuel")
plant_name = mcoe_test.plant_name.unique()[0]
plt.title(f'OpEx for {plant_name}')
plt.ylabel("$/MWh")
plt.ylim(0,100)
plt.legend()
plt.show()
#mcoe_test

## Classify Missing EIA Plants
There are some 2015 EIA 860 units which appear to be missing from the 2017 EIA 860 coal plant data collected above. We need to categorize them:
* Truly missing (the plant and generator unit ID listed in 2015 data do not appear in 2017 data)
* Plant & Unit IDs listed do not correspond to coal plants (fuel_type_code_pudl and energy_source_code_1 are not coal-related)
* The plant is listed as retired in the 2017 data.
* No valid fuel cost data was reported for the years 2012-2017
* No valid generation data was reported for the years 2012-2017
* ??? Other reasons ???

In [ ]:
gens_eia860 = pudl_out.gens_eia860()
missing_gens = (
    pd.read_excel('missing.xlsx').astype({'Gen ID': str}).
    rename(columns={'ORSPL': 'plant_id_eia',
                    'Gen ID': 'generator_id',
                    'Plant Name': 'plant_name',
                    'Utility': 'utility_id_eia',
                    'State': 'state',
                    'Op Status': 'op_status'})
)
existing_gens = gens_eia860[(gens_eia860.report_date.dt.year==end_year)]
shared_gens = pd.merge(missing_gens, existing_gens, on=['plant_id_eia', 'generator_id'], how='left')
categorized_gens = missing_gens.copy().drop(['op_status', 'state'], axis=1)

# Generators we have no record of in the 2017 data
no_2017 = shared_gens[shared_gens.report_date.isnull()][['plant_id_eia', 'generator_id']]
no_2017['no_2017'] = True
categorized_gens = pd.merge(categorized_gens, no_2017, how='left')
categorized_gens['no_2017'] = categorized_gens.no_2017.fillna(False)

# Generators which are not coal
not_coal = shared_gens[shared_gens.fuel_type_code_pudl!='coal'][['plant_id_eia', 'generator_id']]
not_coal['not_coal'] = True
categorized_gens = pd.merge(categorized_gens, not_coal, how='left')
categorized_gens['not_coal'] = categorized_gens.not_coal.fillna(False)

# Generators which are retired.
retired = shared_gens[shared_gens.operational_status_code=='retired'][['plant_id_eia', 'generator_id']]
retired['retired'] = True
categorized_gens = pd.merge(categorized_gens, retired, how='left')
categorized_gens['retired'] = categorized_gens.retired.fillna(False)

# Figure out which plants have no fuel records.
def missing_coal_costs(plant_ids, pudl_out, years=range(start_year,end_year+1)):
    frc_eia923 = pudl_out.frc_eia923()
    missing_costs = []
    for pid in plant_ids:
        recs = frc_eia923[
            (frc_eia923.plant_id_eia==pid) &
            (frc_eia923.fuel_type_code_pudl=='coal') &
            (frc_eia923.report_date.dt.year.isin(years)) &
            (~frc_eia923.total_fuel_cost.isnull())
        ]
        if len(recs)==0:
            missing_costs.append(pid)
    return missing_costs

missing_costs = missing_coal_costs(missing_gens.plant_id_eia.unique(), pudl_out)
missing_costs = missing_gens[(missing_gens.plant_id_eia.isin(missing_costs))]
missing_costs = missing_costs[['plant_id_eia', 'generator_id']]
missing_costs['missing_costs'] = True
categorized_gens = pd.merge(categorized_gens, missing_costs, how='left')
categorized_gens['missing_costs'] = categorized_gens.missing_costs.fillna(False)


def missing_net_gen(missing_gens, pudl_out, years=range(start_year,end_year+1)):
    missing_gens = missing_gens.copy()[['plant_id_eia', 'generator_id']]
    gen_eia923 = pudl_out.gen_eia923()
    recs_to_check = pd.merge(missing_gens, gen_eia923, how='left')
    missing_netgen = []
    for ids in missing_gens.itertuples(index=False):
        recs = recs_to_check[
            (recs_to_check.plant_id_eia==ids.plant_id_eia) &
            (recs_to_check.generator_id==ids.generator_id) &
            (recs_to_check.report_date.dt.year.isin(years)) &
            (~recs_to_check.net_generation_mwh.isnull())
        ]
        if len(recs)==0:
            missing_netgen.append((ids.plant_id_eia, ids.generator_id))
    return pd.DataFrame.from_records(missing_netgen, columns=['plant_id_eia', 'generator_id'])

missing_gen = missing_net_gen(missing_gens, pudl_out)
missing_gen['missing_gen'] = True
categorized_gens = pd.merge(categorized_gens, missing_gen, how='left')
categorized_gens['missing_gen'] = categorized_gens.missing_gen.fillna(False)


uncategorized_gens = categorized_gens[
    (~categorized_gens.no_2017) &
    (~categorized_gens.not_coal) &
    (~categorized_gens.retired) &
    (~categorized_gens.missing_costs) &
    (~categorized_gens.missing_gen)
]
print(f"Uncategorized Generators: {len(uncategorized_gens)}")
uncategorized_gens.drop('#', axis=1).to_csv('uncategorized_missing_gens.csv', index=False)
categorized_gens.drop('#', axis=1).to_csv('categorized_missing_gens.csv', index=False)

In [ ]:
now = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
with pd.ExcelWriter(f'rough_coal_mcoe_{now}.xlsx') as writer:
    mcoe_both.to_excel(writer, sheet_name='FERC and EIA', index=False)
    steam_agg_ferc1.to_excel(writer, sheet_name='FERC Source Data', index=False)
    (
        rolling_df.drop('index', axis=1).
        loc[(rolling_df.plant_id_pudl.isin(plant_ids_2017)) &
            (rolling_df.fuel_type_code_pudl=='coal'),:].
        to_excel(writer, sheet_name='EIA Source Data', index=False)
    )